<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#FUNCTIONAL-QUEUES" data-toc-modified-id="FUNCTIONAL-QUEUES-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>FUNCTIONAL QUEUES</a></span></li><li><span><a href="#INFORMATION-HIDING" data-toc-modified-id="INFORMATION-HIDING-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>INFORMATION HIDING</a></span></li><li><span><a href="#VARIANCE-ANNOTATIONS" data-toc-modified-id="VARIANCE-ANNOTATIONS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>VARIANCE ANNOTATIONS</a></span></li><li><span><a href="#CHECKING-VARIANCE-ANNOTATIONS" data-toc-modified-id="CHECKING-VARIANCE-ANNOTATIONS-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>CHECKING VARIANCE ANNOTATIONS</a></span></li><li><span><a href="#LOWER-BOUNDS" data-toc-modified-id="LOWER-BOUNDS-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>LOWER BOUNDS</a></span></li><li><span><a href="#CONTRAVARIANCE" data-toc-modified-id="CONTRAVARIANCE-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>CONTRAVARIANCE</a></span></li><li><span><a href="#OBJECT-PRIVATE-DATA" data-toc-modified-id="OBJECT-PRIVATE-DATA-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>OBJECT PRIVATE DATA</a></span></li><li><span><a href="#UPPER-BOUNDS" data-toc-modified-id="UPPER-BOUNDS-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>UPPER BOUNDS</a></span></li></ul></div>

# FUNCTIONAL QUEUES

In [1]:
class SlowAppendQueue[T](elems: List[T]) {
    def head = elems.head
    def tail = new SlowAppendQueue(elems.tail)
    def enqueue(x: T) = new SlowAppendQueue(elems ::: List(x))
}

defined class SlowAppendQueue

In [2]:
class SlowHeadQueue[T](smele: List[T]) {
    def head = smele.last
    def tail = new SlowHeadQueue(smele.init)
    def enqueue(x: T) = new SlowHeadQueue(x :: smele)
}

defined class SlowHeadQueue

In [13]:
class Queue[T](
    private val leading: List[T],
    private val trailing: List[T]
) {
    private def mirror = 
        if (leading.isEmpty)
            new Queue(trailing.reverse, Nil)
        else
            this
    
    def head = mirror.leading.head
    
    def tail = {
        val q = mirror
        new Queue(q.leading.tail, q.trailing)
    }
    
    def enqueue(x: T) =
        new Queue(leading, x :: trailing)
    
    override def toString = {
        val q = mirror
        "Queue("+(leading ++ trailing.reverse).mkString(",")+")"
    }
}

object Queue {
    def apply[T](xs: T*) = new Queue(xs.toList, Nil)
}

defined class Queue
defined object Queue

In [14]:
val q = Queue(1,2,3)
val q1 = q enqueue 4
q1

q: Queue[Int] = Queue(1,2,3)
q1: Queue[Int] = Queue(1,2,3,4)
res13_2: Queue[Int] = Queue(1,2,3,4)

# INFORMATION HIDING

In [17]:
trait Queue[T] {
    def head: T
    def tail: Queue[T]
    def enqueue(x: T): Queue[T]
}

object Queue {
    def apply[T](xs: T*): Queue[T] = new QueueImpl[T](xs.toList, Nil)
    
    private class QueueImpl[T](
        private val leading: List[T],
        private val trailing: List[T]
    ) extends Queue[T] {
        private def mirror = 
            if (leading.isEmpty)
                new QueueImpl(trailing.reverse, Nil)
            else
                this

        def head = mirror.leading.head

        def tail = {
            val q = mirror
            new QueueImpl(q.leading.tail, q.trailing)
        }

        def enqueue(x: T) =
            new QueueImpl(leading, x :: trailing)

        override def toString = {
            val q = mirror
            "Queue("+(leading ++ trailing.reverse).mkString(",")+")"
        }
    }
}

defined trait Queue
defined object Queue

In [18]:
val q = Queue(1,2,3)
val q1 = q enqueue 4
q1

q: Queue[Int] = Queue(1,2,3)
q1: Queue[Int] = Queue(1,2,3,4)
res17_2: Queue[Int] = Queue(1,2,3,4)

# VARIANCE ANNOTATIONS

```scala
sealed abstract class List[+A] extends AbstractSeq[A]
                                  with LinearSeq[A]
                                  with Product
                                  with GenericTraversableTemplate[A, List]
                                  with LinearSeqOptimized[A, List[A]]
                                  with scala.Serializable {}


trait Function1[@specialized(scala.Int, scala.Long, scala.Float, scala.Double) -T1, 
                @specialized(scala.Unit, scala.Boolean, scala.Int, scala.Float, scala.Long, scala.Double) +R] 
extends AnyRef {}

trait Function2[@specialized(scala.Int, scala.Long, scala.Double) -T1, 
                @specialized(scala.Int, scala.Long, scala.Double) -T2, 
                @specialized(scala.Unit, scala.Boolean, scala.Int, scala.Float, scala.Long, scala.Double) +R] 
extends AnyRef {}
```

# CHECKING VARIANCE ANNOTATIONS

# LOWER BOUNDS

In [19]:
class Queue[+T](
    private val leading: List[T],
    private val trailing: List[T]
) {
    def eneuque[U >: T](x: U) = 
        new Queue[U](leading, x :: trailing)
}

defined class Queue

# CONTRAVARIANCE

In [2]:
class Publication(val title: String)
class Book(title: String) extends Publication(title)

object Library {
val books: Set[Book] =
    Set(
        new Book("Programming in Scala"),
        new Book("Walden")
     )
    def printBookList(info: Book => AnyRef) = {
        for (book <- books) println(info(book))
    }
}
 
object Customer extends App {
    def getTitle(p: Publication): String = p.title
    Library.printBookList(getTitle)
}

Library.printBookList(Customer.getTitle)

Programming in Scala
Walden


defined class Publication
defined class Book
defined object Library
defined object Customer

In [3]:
// Book => AnyRef
val f1: Book => AnyRef = _.title

// Publication => String
val f2: Publication => String = _.title

f2.isInstanceOf[Book => AnyRef]

f1: Book => AnyRef = ammonite.$sess.cmd2$Helper$$Lambda$2229/522601678@2e8c1bfd
f2: Publication => String = ammonite.$sess.cmd2$Helper$$Lambda$2230/1727888692@728af151
res2_2: Boolean = true

# OBJECT PRIVATE DATA

In [4]:
class Queue[+T] private (
    private[this] var leading: List[T],
    private[this] var trailing: List[T]
) {
    private def mirror() = 
        if (leading.isEmpty) {
            while (!trailing.isEmpty) {
                leading = trailing.head :: leading
                trailing = trailing.tail
            }
        }
    
    def head: T = {
        mirror()
        leading.head
    }
    
    def tail: Queue[T] = {
        mirror()
        new Queue(leading.tail, trailing)
    }
    
    def eneuque[U >: T](x: U) =
        new Queue[U](leading, x :: trailing)
}

defined class Queue

# UPPER BOUNDS

In [6]:
class Person(val firstName: String, val lastName: String) extends Ordered[Person] {

    def compare(that: Person) = {
        val lastNameComparison = lastName.compareToIgnoreCase(that.lastName)
        if (lastNameComparison != 0)
            lastNameComparison
        else
            firstName.compareToIgnoreCase(that.firstName)
    }
    
    override def toString = firstName + " " + lastName
}

def orderedMergeSort[T <: Ordered[T]](xs: List[T]): List[T] = {
    def merge(xs: List[T], ys: List[T]): List[T] = 
        (xs, ys) match {
            case (Nil, _) => ys
            case (_, Nil) => xs
            case (x :: xs1, y :: ys1) =>
                if (x < y) x :: merge(xs1, ys)
                else y :: merge(xs, ys1)
        }
    
    val n = xs.length / 2
    if (n == 0) xs
    else {
        val (ys, zs) = xs splitAt n
        merge(orderedMergeSort(ys), orderedMergeSort(zs))
    }
}

val people = List(
    new Person("Larry", "Wall"),
    new Person("Anders", "Hejlsberg"),
    new Person("Guido", "van Rossum"),
    new Person("Alan", "Kay"),
    new Person("Yukihiro", "Matsumoto")
)
val sortedPeople = orderedMergeSort(people)


defined class Person
defined function orderedMergeSort
people: List[Person] = List(
  Larry Wall,
  Anders Hejlsberg,
  Guido van Rossum,
  Alan Kay,
  Yukihiro Matsumoto
)
sortedPeople: List[Person] = List(
  Anders Hejlsberg,
  Alan Kay,
  Yukihiro Matsumoto,
  Guido van Rossum,
  Larry Wall
)